In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx

import pandas as pd

import copy

import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.data import Batch, Data, Dataset

from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.preprocessing import MinMaxScaler
import time

import math


In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!python --version

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("No GPU detected.")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
files = ["donken",
         "Holmgrens",
         "IONITY",
         "Jureskogs_Vattenfall",
         "UFC"]

nr_of_data_points = 163618
splits=[0.8, 0.9]

train_data = {}
val_data = {}
test_data = {}

for f in files:
    data = pd.read_csv('data_' + f + '_5T_k-10.csv')
    #data.info()
    data.set_index('Unnamed: 0', inplace=True)
    
    data = data.drop(columns=data.columns.difference(['Occupancy']))
    
    train_data[f] = data[:int(splits[0]*nr_of_data_points)]
    val_data[f] = data[int(splits[0]*nr_of_data_points):int(splits[1]*nr_of_data_points)]
    test_data[f] = data[int(splits[1]*nr_of_data_points):]

In [ ]:
# Create a graph
G = nx.Graph()

# Add nodes
num_nodes = 5
G.add_nodes_from(range(num_nodes))

Jureskogs_Vattenfall = 0
IONITY = 1
donken = 2
Holmgrens = 3
UFC = 4

# Define edges to connect specific nodes with custom weights
edges_to_connect = [
    (Jureskogs_Vattenfall, IONITY, 230),
    (Jureskogs_Vattenfall, UFC, 750),
    (Jureskogs_Vattenfall, Holmgrens, 750),
    (Jureskogs_Vattenfall, donken, 650),
    (IONITY, UFC, 550),
    (IONITY, Holmgrens, 500),
    (IONITY, donken, 450),
    (UFC, Holmgrens, 280),
    (UFC, donken, 550),
    (Holmgrens, donken, 550)
]

# Add edges with custom weights
for edge in edges_to_connect:
    G.add_edge(edge[0], edge[1], weight=edge[2])

# Create adjacency matrix with weights
adj_matrix = nx.adjacency_matrix(G).todense()

torch_adj_matrix = torch.Tensor(adj_matrix)

edge_index = torch_adj_matrix.nonzero(as_tuple=False).t().contiguous()
edge_attr = torch_adj_matrix[torch_adj_matrix.nonzero()].reshape(-1, 1)


# Define colors for nodes
node_colors = ['Yellow'] * num_nodes

print(G.nodes)

# Draw the graph
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=700, font_size=10)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Show the graph
plt.show()


In [ ]:

class datasetMaker(Dataset):
    def __init__(self, station_data, edge_index, edge_attr, seq_len, future_steps, batch_size):
        self.station_data = station_data
        self.size = station_data["donken"].shape[0]
        self.edge_index = edge_index
        self.edge_attr = edge_attr
        self.seq_len = seq_len
        self.future_steps = future_steps
        self.batch_size = batch_size

    def __len__(self):
        return self.size - self.seq_len - self.future_steps

    def __getitem__(self, index):
        
        seq_end = index + self.seq_len
        fut_end = index + self.seq_len + self.future_steps
        
        node_features = []
        for i, (station, data) in enumerate(self.station_data.items()):
            node_feature = data.iloc[index:seq_end].values
            node_features.append(node_feature)
        node_features = torch.tensor(np.array(node_features)).float()

        labels = []
        for i, (station, data) in enumerate(self.station_data.items()):
            label = data.iloc[seq_end:fut_end].values
            labels.append(label)
        labels = torch.unsqueeze(torch.tensor(np.array(labels)), dim=2)
        
        Gdata = Data(x=node_features, y=labels, edge_index=self.edge_index, edge_attr=self.edge_attr)

        return Gdata, labels

    
def custom_collate(batch):
    label = torch.cat([i[1] for i in batch])
    
    label = label.squeeze(3)
    
    batch = Batch.from_data_list([b[0] for b in batch])

    
    return batch, label


future_steps = 36
seq_len = 576
batch_size = 64

train_dataset = datasetMaker(train_data, edge_index, edge_attr, seq_len, future_steps, batch_size)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=custom_collate)

val_dataset = datasetMaker(val_data, edge_index, edge_attr, seq_len, future_steps, batch_size)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=custom_collate)

test_dataset = datasetMaker(test_data, edge_index, edge_attr, seq_len, future_steps, batch_size)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=custom_collate)


print("train len ", len(train_loader))
print("val len   ", len(val_loader))
print("test len  ", len(test_loader))


for data, label in train_loader:
    print(data)
    print(label.shape)
    break



In [ ]:

def reshape_to_batches(x, batch_description):
    """
        Does something like this:
        torch.Size([28, 576, 64]) --> torch.Size([4, 7, 576, 64])
    """
    num_splits = batch_description.max().item() + 1
    new_shape_dim_0 = num_splits
    new_shape_dim_1 = x.size(0) // new_shape_dim_0
    new_shape = torch.Size([new_shape_dim_0, new_shape_dim_1] + list(x.size()[1:]))
    reshaped_tensor = x.view(new_shape)
    return reshaped_tensor


class GCN(torch.nn.Module):
    def __init__(self, in_channels=1, gcn_hidden_channels=8, gcn_layers=1):
        super(GCN, self).__init__()
        self.in_conv = GCNConv(in_channels, gcn_hidden_channels)
        self.hidden_convs = [GCNConv(gcn_hidden_channels, gcn_hidden_channels).cuda() for i in range(gcn_layers - 1)]

    def forward(self, x, edge_index, batch):
        x = x.float()
        x = self.in_conv(x, edge_index)
        for conv in self.hidden_convs:
            x = F.relu(x)
            x = conv(x, edge_index)
        x = F.relu(x)
        return x

class SimpleTransformer(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, nhead, seq_length, num_layers=1, dropout=0.1):
        super(SimpleTransformer, self).__init__()

        self.seq_length = seq_length
        self.output_size = output_size
        self.hidden_layer_size = hidden_layer_size
        
        self.embeddingIn = nn.Linear(input_size, hidden_layer_size)
        self.embeddingTGT = nn.Linear(output_size, hidden_layer_size)
        
        self.PositionalEncoding = PositionalEncoding(max_len=1000, d_model=hidden_layer_size)
        
        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_layer_size, nhead=nhead, 
                                                    dim_feedforward=4*hidden_layer_size, dropout=dropout, 
                                                    activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=encoder_layers, num_layers=num_layers)
        # tr
        decoder_layers = nn.TransformerDecoderLayer(d_model=hidden_layer_size, nhead=nhead,
                                                    dim_feedforward=4*hidden_layer_size, dropout=dropout, 
                                                    activation='gelu')
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer=decoder_layers, num_layers=num_layers)

        self.linear1 = nn.Linear(hidden_layer_size, output_size)
                
    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
        return mask
        
    def forward(self, x, tgt=None, last_value=None, inference=False):
        last_value = torch.unsqueeze(last_value, dim=2)

        initial_tgt = last_value#x[:, -1:]
        #start_value = x[:, -1:]
        
        tgt_input = torch.cat([last_value, tgt[:, :-1]], dim=1)
        
        x = self.embeddingIn(x)
        x = self.PositionalEncoding(x)
        enc_mask = self.generate_square_subsequent_mask(x.size(1)).to(tgt.device)
        x = x.permute(1, 0, 2)
        encoder_output = self.transformer_encoder(x, mask=enc_mask)
        encoder_output = encoder_output.permute(1, 0, 2)
        
        if inference:
            tgt_gen = initial_tgt
            #print(encoder_output.shape)
            #encoder_output = encoder_output.permute(1, 0, 2)
            #print(encoder_output.shape)
            #print(tgt_gen.shape)
            generated_sequence = torch.zeros((initial_tgt.size(0), self.seq_length, self.output_size), device=x.device)
            encoder_output = encoder_output.permute(1,0,2)

            for i in range(self.seq_length):
                #print(tgt_gen.shape)
                
                tgt_emb = self.embeddingTGT(tgt_gen)
                #print(tgt_emb.shape)
                
                tgt_emb = self.PositionalEncoding(tgt_emb)
                tgt_emb = tgt_emb.permute(1, 0, 2)
                #print(tgt_emb.shape)

                decoder_output = self.transformer_decoder(tgt_emb, encoder_output)

                output_step = self.linear1(decoder_output[-1, :, :])
                output_step = output_step.unsqueeze(1) 

                generated_sequence[:, i:i+1, :] = output_step

                tgt_gen = torch.cat((tgt_gen, output_step), dim=1)
                #start_value = torch.unsqueeze(x[:, -1:, 1], 1)

                if tgt_gen.size(1) > self.seq_length:
                    tgt_gen = tgt_gen[:, 1:, :]

            return generated_sequence

        else:
            tgt = self.embeddingTGT(tgt_input)
            tgt = self.PositionalEncoding(tgt)
            tgt = tgt.permute(1, 0, 2)

            tgt_mask = self.generate_square_subsequent_mask(tgt.size(0)).to(tgt.device)

            encoder_output = encoder_output.permute(1,0,2)
            
            decoder_output = self.transformer_decoder(tgt, encoder_output, tgt_mask=tgt_mask)
            #try dropout here
            output = self.linear1(decoder_output)

            return output.permute(1, 0, 2)
        
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Correct the shaping of pe to [1, max_len, d_model]
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        #print(x.shape)
        #print(self.pe[:, :x.size(1), :].shape)
        # Adjust slicing of pe to match the sequence length of x
        # pe is broadcasted correctly across the batch dimension
        return x + self.pe[:, :x.size(1), :]

class STGCN(nn.Module):
    
    def __init__(self, in_channels, gcn_layers, hidden_channels, transformer_hidden_size, transformer_num_layers, transformer_nhead, out_channels):
        super(STGCN, self).__init__()
        print("\033[100mhidden_channels:", hidden_channels,
              "   GCN hidden layers:", gcn_layers,
              "   transformer_hidden_size:", transformer_hidden_size,
              "   transformer_num_layers:", transformer_num_layers,
              "   transformer_nhead:", transformer_nhead, "\033[0m")

        self.GCN = GCN(in_channels=in_channels, gcn_hidden_channels=hidden_channels, gcn_layers=gcn_layers)

        self.transformer = SimpleTransformer(input_size = hidden_channels, hidden_layer_size=transformer_hidden_size,
                                             output_size=out_channels, seq_length=36, num_layers=transformer_num_layers,
                                             nhead=transformer_nhead).cuda()
        
    def forward(self, data, inference=False):    
        batch = data.batch
        label = data.y
        label = torch.squeeze(label, 2)
        
        data.x = data.x.float()  # Convert node features to Double
        data.edge_attr = data.edge_attr.float()  # Convert edge attributes to Double
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
       
        # Spatial processing
        x = self.GCN(x, edge_index, edge_attr)

        x = reshape_to_batches(x, batch)
        last_value = reshape_to_batches(data.x[:,-1,:],batch)
        label = reshape_to_batches(label, batch)
                
        # Reshape and pass data through the model for each station
        predictions = []
       
        for station_data, station_label, station_last_value in zip(x.permute(1,0,2,3), label.permute(1,0,2,3), last_value.permute(1,0,2)):
            output = self.transformer(station_data, station_label, station_last_value, inference)
            predictions.append(output)

        # Concatenate predictions for all stations
        predictions = torch.stack(predictions, dim=1)
        return predictions

# Example usage:
# Define the adjacency matrix for spatial processing (A_spatial)
# Define the input size, number of layers, and number of heads for the temporal transformer


In [ ]:
class MultiStepLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(MultiStepLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Flatten LSTM parameters
        self.lstm.flatten_parameters()
        
        # Fully connected layer to map LSTM output to desired output_size
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, future=1):
        
        predictions = []
        
        
        for _ in range(future):
            # Initialize hidden state and cell state        
            batch_size, sequence_length, _ = x.size()
            h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
            c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

            # LSTM forward pass
            out, (h0, c0) = self.lstm(x, (h0, c0))
            
            pred = out[:, -1, :]
            
            x = torch.cat((x, pred.unsqueeze(1)), dim=1)
            
            t = self.fc(pred)
            predictions.append(t) # Append occupancy to predictions
            

        # Stack predictions along the sequence length dimension'
        predictions = torch.cat(predictions, dim=1)
        
        predictions = torch.unsqueeze(predictions, dim = 2)
        return predictions
    
class STGCNLSTM(nn.Module):
    
    def __init__(self, in_channels, gcn_layers, hidden_channels, lstm_layers, out_channels):
        super(STGCN, self).__init__()
        
        print("\033[100mhidden_channels:", hidden_channels,
              "   GCN hidden layers:", gcn_layers,
              "   lstm_layers:", lstm_layers, "\033[0m")

        
        self.GCN = GCN(in_channels=in_channels, gcn_hidden_channels=hidden_channels, gcn_layers=gcn_layers)

        self.lstm = MultiStepLSTM(hidden_channels, hidden_channels, out_channels, lstm_layers)
        
    def forward(self, data):    
 
        batch = data.batch
        
        data.x = data.x.float()  # Convert node features to Double
        data.edge_attr = data.edge_attr.float()  # Convert edge attributes to Double
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
       
        # Spatial processing
        x = self.GCN(x, edge_index, edge_attr)

        x = reshape_to_batches(x, batch)
        # Reshape and pass data through the model for each station
        predictions = []
        for station_data in x.permute(1,0,2,3):  # Iterate over each station
            #station_data = station_data.permute(1, 0, 2)  # Reshape for LSTM (batch_first=True)
            output = self.lstm(station_data, future=future_steps)
            predictions.append(output)

        # Concatenate predictions for all stations
        predictions = torch.stack(predictions, dim=1)
        return predictions

# Loading model ST-GCN LSTM

In [ ]:
best_model = STGCNLSTM

# Load the state dictionary
best_model.load_state_dict(torch.load('best_ST-GCN_model_direct-connect-transformer_final.pth'))

# Set the model to evaluation mode if you are testing (not training)
best_model.eval()

# Loading model ST-GCN transformer

In [ ]:
best_model = STGCN(in_channels=1, gcn_layers=2, hidden_channels=2, transformer_hidden_size=12,
                  transformer_num_layers=1, transformer_nhead=3, out_channels=1).cuda()

# Load the state dictionary
best_model.load_state_dict(torch.load('best_ST-GCN_model_direct-connect-transformer_final.pth'))

# Set the model to evaluation mode if you are testing (not training)
best_model.eval()

# Statistics

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import warnings

# Suppress warnings to keep the output clean
warnings.filterwarnings("ignore")

# Assuming val_loader is your DataLoader and best_model is your model

# Initialize storage for predictions and true values for all models
all_predictions = []  # ST-GCN predictions
all_labels = []  # ST-GCN labels
all_arima_forecasts = []  # MA(40) ARIMA predictions
all_ar_forecasts = []  # AR(60) predictions
all_arima_36_forecasts = []  # ARIMA(36, 1, 36) predictions
all_last_value_forecasts = []  # Last known value predictions

batch_count = 0  # Number of batches to process
sequence_count = 0  # Track the total number of processed sequences

for data, label in test_loader:
    
    label = reshape_to_batches(label, data.batch)
    label = label.float().cpu()
    data = data.cuda()

    # Get predictions from the ST-GCN model
    predictions = best_model(data, inference=True)
    
    # Convert tensors to numpy arrays
    predictions = predictions.detach().cpu().numpy()
    label = label.numpy()
    
    # Iterate over each sequence in the batch
    for i in range(label.shape[0]):
        if sequence_count >= 16:  # Stop after 16 predictions
            break

        actual_data = label[i, 2, :, 0]  # Actual data used for all models
        all_predictions.append(predictions[i, 2, :, 0])  # ST-GCN predictions
        all_labels.append(actual_data)  # True labels

        # MA(40) ARIMA model
        ma_model = ARIMA(actual_data, order=(0, 0, 40))
        ma_model_fitted = ma_model.fit()
        ma_forecast = ma_model_fitted.forecast(steps=len(actual_data))
        all_arima_forecasts.append(ma_forecast)
        
        # AR(60) model
        ar_model = ARIMA(actual_data, order=(60, 0, 0))
        ar_model_fitted = ar_model.fit()
        ar_forecast = ar_model_fitted.forecast(steps=len(actual_data))
        all_ar_forecasts.append(ar_forecast)

        # ARIMA(36, 1, 36) model
        arima_36_model = ARIMA(actual_data, order=(36, 1, 36))
        arima_36_model_fitted = arima_36_model.fit()
        arima_36_forecast = arima_36_model_fitted.forecast(steps=len(actual_data))
        all_arima_36_forecasts.append(arima_36_forecast)

        # Last known value prediction
        last_value = actual_data[-1]
        last_value_forecast = np.full(len(actual_data), last_value)
        all_last_value_forecasts.append(last_value_forecast)
        
        sequence_count += 1
    
    batch_count += 1
    if sequence_count >= 16:
        break

# Plotting the predictions
fig, axes = plt.subplots(4, 4, figsize=(20, 20))  # 4 rows and 4 columns for 16 sequences
axes = axes.flatten()

for i in range(16):
    axes[i].plot(all_predictions[i], label="ST-GCN Predictions", color="blue")
    axes[i].plot(all_arima_forecasts[i], label="MA(40) ARIMA Predictions", color="green")
    axes[i].plot(all_ar_forecasts[i], label="AR(60) Predictions", color="red")
    axes[i].plot(all_arima_36_forecasts[i], label="ARIMA(36, 1, 36) Predictions", color="purple")
    axes[i].plot(all_last_value_forecasts[i], label="Last Known Value", color="gray", linestyle='--')
    axes[i].plot(all_labels[i], label="True Values", color="orange")
    axes[i].set_title(f"Prediction {i+1}")
    axes[i].legend()
    axes[i].set_ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, median_absolute_error

# Define metrics calculation functions
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def r_squared(y_true, y_pred):
    return r2_score(y_true, y_pred)

best_model.eval()

# Data containers with explicit tracking
metrics_data = {'station': files, 'mse': [], 'rmse': [], 'mae': [], 'r2': [], 'msle': [], 'medae': []}
for metric in ['mse', 'rmse', 'mae', 'r2', 'msle', 'medae']:
    metrics_data[metric] = [[] for _ in files]  # List of lists for each station

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_labels = reshape_to_batches(batch_labels, batch_data.batch)
        batch_data = batch_data.cuda()
        batch_labels = batch_labels.cuda().float()
        batch_predictions = best_model(batch_data, inference=True)

        batch_predictions = batch_predictions.view(-1, len(files), 36)  # Assume '36' prediction steps
        batch_labels = batch_labels.view(-1, len(files), 36)

        for idx, station_name in enumerate(files):
            p = batch_predictions[:, idx, :].cpu().numpy()
            l = batch_labels[:, idx, :].cpu().numpy()

            # Calculate metrics for each station and append
            mse = mean_squared_error(l, p)
            metrics_data['mse'][idx].append(mse)
            metrics_data['rmse'][idx].append(root_mean_squared_error(l, p))
            metrics_data['mae'][idx].append(mean_absolute_error(l, p))
            metrics_data['r2'][idx].append(r_squared(l, p))
            metrics_data['msle'][idx].append(mean_squared_log_error(l, p))
            metrics_data['medae'][idx].append(median_absolute_error(l, p))

# Average metrics across batches
for metric in ['mse', 'rmse', 'mae', 'r2', 'msle', 'medae']:
    metrics_data[metric] = [np.mean(vals) for vals in metrics_data[metric]]

# Plotting setup
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
colors = plt.cm.viridis(np.linspace(0, 1, len(files)))

for ax, metric in zip(axes.flatten(), ['mse', 'rmse', 'mae', 'r2', 'msle', 'medae']):
    ax.bar(metrics_data['station'], metrics_data[metric], color=colors)
    ax.set_title(f'{metric.upper()} by Station')
    ax.set_xlabel('Station')
    ax.set_ylabel(metric.upper())
    ax.set_xticks(metrics_data['station'])  # Ensure x-ticks correspond to stations
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.scatter([12, 24, 36, 48], [0.0035, 0.0070, 0.0105, 0.0140], color='orange', label='AsynST-GCN', marker='*', s=200)
plt.scatter([12, 24, 36, 48], [0.0041, 0.0083, 0.0098, 0.0177], color='red', label='ST-GCN (Transformer)', marker='*', s=200)
plt.scatter([12, 24, 36, 48], [0.0050, 0.0100, 0.0150, 0.0200], color='blue', label='ST-GCN (LSTM)', marker='*', s=200)
plt.scatter([12, 24, 36, 48], [0.0020, 0.0040, 0.0060, 0.0080], color='darkgreen', label='LSTM', marker='*', s=200)
plt.scatter([12, 24, 36, 48], [0.0045, 0.0090, 0.0135, 0.0180], color='limegreen', label='Transformer', marker='*', s=200)
plt.scatter([12, 24, 36, 48], [0.0060, 0.0120, 0.0180, 0.0240], color='cyan', label='ARIMA', marker='*', s=200)
plt.scatter([12, 24, 36, 48], [0.0030, 0.0060, 0.0090, 0.0120], color='purple', label='MA', marker='*', s=200)
plt.scatter([12, 24, 36, 48], [0.0055, 0.0110, 0.0165, 0.0220], color='black', label='AR', marker='*', s=200)

plt.plot([12, 24, 36, 48], [0.0035, 0.0070, 0.0105, 0.0140], color='orange', linestyle='--')  
plt.plot([12, 24, 36, 48], [0.0041, 0.0083, 0.0098, 0.0177], color='red', linestyle='--')  
plt.plot([12, 24, 36, 48], [0.0050, 0.0100, 0.0150, 0.0200], color='blue', linestyle='--')  
plt.plot([12, 24, 36, 48], [0.0020, 0.0040, 0.0060, 0.0080], color='darkgreen', linestyle='--')  
plt.plot([12, 24, 36, 48], [0.0045, 0.0090, 0.0135, 0.0180], color='limegreen', linestyle='--')  
plt.plot([12, 24, 36, 48], [0.0060, 0.0120, 0.0180, 0.0240], color='cyan', linestyle='--')  
plt.plot([12, 24, 36, 48], [0.0030, 0.0060, 0.0090, 0.0120], color='purple', linestyle='--')  
plt.plot([12, 24, 36, 48], [0.0055, 0.0110, 0.0165, 0.0220], color='black', linestyle='--')  

plt.text(12, 0.002, '1 hour', ha='center', fontsize=10)
plt.text(24, 0.002, '2 hours', ha='center', fontsize=10)
plt.text(36, 0.002, '3 hours', ha='center', fontsize=10)
plt.text(48, 0.002, '4 hours', ha='center', fontsize=10)

plt.title('Forecasting Horizon Comparison')
plt.xlabel('Forecasting window (5-minute data points)')
plt.ylabel('MSE')

plt.xlim(0, 60)
plt.ylim(0, 0.02)

plt.legend()

plt.grid(True)
plt.show()


In [ ]:
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

all_mse_by_step = {station: [[] for _ in range(36)] for station in files}  # Dictionary to store MSE values for each station and prediction step
num_steps = 36  # Total number of prediction steps

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_labels = reshape_to_batches(batch_labels, batch_data.batch)
        batch_data = batch_data.cuda()
        batch_labels = batch_labels.cuda().float()

        batch_predictions = best_model(batch_data, inference=True)

        for station_index, station_name in enumerate(files):
            for step in range(num_steps):
                mse_step = F.mse_loss(batch_predictions[:, station_index, step], batch_labels[:, station_index, step]).item()
                all_mse_by_step[station_name][step].append(mse_step)

mean_mse_by_step = {station: [np.mean(step_mse_values) for step_mse_values in all_mse_by_step[station]] for station in files}

plt.figure(figsize=(14, 8))
for station in files:
    plt.plot(range(1, num_steps + 1), mean_mse_by_step[station], marker='o', label=station)

plt.title('Mean MSE for Each Prediction Step Across Sequences by Station')
plt.xlabel('Prediction Step')
plt.ylabel('Mean MSE')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Set the model to evaluation mode
best_model.eval()

# Data containers for each station
station_data = {name: {'predictions': [], 'labels': []} for name in files}

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        # Assuming your reshape_to_batches handles batch data appropriately
        batch_labels = reshape_to_batches(batch_labels, batch_data.batch)
        batch_data = batch_data.cuda()
        batch_labels = batch_labels.cuda().float()

        # Obtain predictions
        batch_predictions = best_model(batch_data, inference=True)
        
        batch_predictions = batch_predictions.view(-1, len(files), 36)  # Assume '36' prediction steps
        batch_labels = batch_labels.view(-1, len(files), 36)

        for idx, station_name in enumerate(files):
            # Extract predictions and actuals for one station
            predictions = batch_predictions[:, idx, :].flatten().cpu().numpy()
            labels = batch_labels[:, idx, :].flatten().cpu().numpy()

            # Store data for plotting
            station_data[station_name]['predictions'].extend(predictions)
            station_data[station_name]['labels'].extend(labels)

# Plot predicted vs actual values for each station
fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(15, 10))  # Adjust subplot layout to 1 row
for ax, station_name in zip(axs, files):
    preds = np.array(station_data[station_name]['predictions'])
    actuals = np.array(station_data[station_name]['labels'])

    lims = [0, 1]
    
    ax.scatter(preds, actuals, alpha=0.1)
    ax.plot(lims, lims, 'r--', lw=2)  # Change reference line to red
    ax.set_aspect('equal', adjustable='box')  # Ensure that axes are equally scaled
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_title(f'Predicted vs Actual for {station_name}')
    ax.set_xlabel('Predicted Values')
    ax.set_ylabel('Actual Values')

plt.tight_layout()
plt.show()

# Random predictions

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Data containers for each station
station_data = {name: {'predictions': [], 'labels': []} for name in files}

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        # Assuming your reshape_to_batches handles batch data appropriately
        batch_labels = reshape_to_batches(batch_labels, batch_data.batch)
        batch_data = batch_data.cuda()
        batch_labels = batch_labels.cuda().float()

        # Generate random predictions that match the shape of batch_labels
        batch_predictions = np.random.rand(*batch_labels.size()).astype(np.float32)  # Ensure it's float32 to match typical PyTorch dtype

        for idx, station_name in enumerate(files):
            # Extract random predictions and actuals for one station
            predictions = batch_predictions[:, idx, :].flatten()
            labels = batch_labels[:, idx, :].flatten().cpu().numpy()

            # Store data for plotting
            station_data[station_name]['predictions'].extend(predictions)
            station_data[station_name]['labels'].extend(labels)

# Plot predicted vs actual values for each station
fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(15, 10))  # Adjust subplot layout to 1 row
for ax, station_name in zip(axs, files):
    preds = np.array(station_data[station_name]['predictions'])
    actuals = np.array(station_data[station_name]['labels'])

    lims = [0, 1]
    
    ax.scatter(preds, actuals, alpha=0.1)
    ax.plot(lims, lims, 'r--', lw=2)  # Change reference line to red
    ax.set_aspect('equal', adjustable='box')  # Ensure that axes are equally scaled
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_title(f'Predicted vs Actual for {station_name}')
    ax.set_xlabel('Predicted Values')
    ax.set_ylabel('Actual Values')

plt.tight_layout()
plt.show()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

# Set the model to evaluation mode
best_model.eval()

# Data containers for each station
station_data = {name: {'predictions': [], 'labels': []} for name in files}

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        # Assuming your reshape_to_batches handles batch data appropriately
        batch_labels = reshape_to_batches(batch_labels, batch_data.batch)
        batch_data = batch_data.cuda()
        batch_labels = batch_labels.cuda().float()

        # Generate random predictions that match the shape of batch_labels
        batch_predictions = torch.rand_like(batch_labels)  # Generate random predictions directly as a torch tensor

        for idx, station_name in enumerate(files):
            # Extract random predictions and actuals for one station
            predictions = batch_predictions[:, idx, :].flatten()
            labels = batch_labels[:, idx, :].flatten()

            # Store data for plotting and MSE calculation
            station_data[station_name]['predictions'].extend(predictions.cpu().numpy())
            station_data[station_name]['labels'].extend(labels.cpu().numpy())

# Calculate and print MSE for each station
for station_name in files:
    preds = torch.tensor(station_data[station_name]['predictions'])
    actuals = torch.tensor(station_data[station_name]['labels'])

    mse = F.mse_loss(preds, actuals)
    print("Mean Squared Error for", station_name, ":\t", mse.item())